In [1]:
# This tests PynPoint

# created 2018 Sept. 20 by E.S.

In [1]:
import urllib
import numpy as np
import matplotlib.pyplot as plt

import PynPoint

from PynPoint import Pypeline
from PynPoint.IOmodules.Hdf5Reading import Hdf5ReadingModule
from PynPoint.ProcessingModules import PSFpreparationModule, \
                                       PcaPsfSubtractionModule

In [2]:
working_place = "./pynpoint_experimentation/working_place/"
input_place = "./pynpoint_experimentation/input_place/"
output_place = "./pynpoint_experimentation/output_place/"

url = urllib.URLopener()
url.retrieve("https://people.phys.ethz.ch/~stolkert/BetaPic_NACO_Mp.hdf5",
             input_place+"BetaPic_NACO_Mp.hdf5")

pipeline = Pypeline(working_place_in=working_place,
                    input_place_in=input_place,
                    output_place_in=output_place)

Initiating PynPoint... [DONE]


In [3]:
read = Hdf5ReadingModule(name_in="read",
                         input_filename="BetaPic_NACO_Mp.hdf5",
                         input_dir=None,
                         tag_dictionary={"stack":"stack"})

pipeline.add_module(read)

In [4]:
prep = PSFpreparationModule(name_in="prep",
                            image_in_tag="stack",
                            image_out_tag="prep",
                            image_mask_out_tag=None,
                            mask_out_tag=None,
                            norm=False,
                            resize=None,
                            cent_size=0.15,
                            edge_size=1.1)

pipeline.add_module(prep)

In [5]:
pca = PcaPsfSubtractionModule(pca_numbers=(20, ),
                              name_in="pca",
                              images_in_tag="prep",
                              reference_in_tag="prep",
                              res_mean_tag="residuals")

pipeline.add_module(pca)

In [6]:
pipeline.run()

Validating Pypeline... [DONE]
Running Hdf5ReadingModule... [DONE]
Running PSFpreparationModule... [DONE]
Constructing PSF model... [DONE]
Creating residuals... [DONE]


In [7]:
residuals = pipeline.get_data("residuals")
pixscale = pipeline.get_attribute("stack", "PIXSCALE")

size = pixscale*np.size(residuals, 1)/2.

In [10]:
plt.imshow(residuals[0, ], origin='lower', extent=[size, -size, -size, size])
plt.title("beta Pic b - NACO M' - mean residuals")
plt.xlabel('R.A. offset [arcsec]', fontsize=12)
plt.ylabel('Dec. offset [arcsec]', fontsize=12)
plt.colorbar()
#plt.show()
plt.savefig(output_place+"residuals.png")

In [13]:
pipeline